<a href="https://colab.research.google.com/github/ahmadluay9/ADK-Training/blob/main/06_sql_multi_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# 📊 Multi-Agent SQL Pipeline Architecture

This notebook demonstrates a multi-agent AI workflow designed to translate natural language into validated SQL queries.

**Key Features:**
* **Parallel Context Retrieval:** Fetches schema and past query examples.
* **Self-Healing Code:** Uses an automated, iterative validation loop (up to 3x) to fix SQL errors before execution.
* **Natural Language Insights:** Translates raw database rows back into human-readable answers.


The following diagram maps out a multi-agent AI system designed to translate user questions into database queries, execute them, and return natural language answers.

Here is the concise breakdown of the pipeline:

* **1. Triage (Root Agent Decision):** The system first evaluates the user's input. If it is standard conversation, it replies directly. If it requires data, it triggers the SQL pipeline.

* **2. Context Gathering (Retrieval Stage):** Two agents work in parallel to fetch the necessary context—one grabs the database rules (Schema) and the other finds relevant past queries (SQL Retrieval).

* **3. Query Drafting (SQL Generation):** A dedicated agent uses the gathered schema and past examples to write an initial SQL query.

* **4. Self-Correction (Validation Loop):** The drafted query is tested. If it has errors, a refiner agent attempts to fix it. This cycle repeats up to 3 times to ensure the code is functional before running it.

* **5. Execution (SQL Executor):** The error-free, validated SQL is executed against the actual database to retrieve the raw data.

* **6. Delivery (Insight Generator):** A final agent interprets those raw database results and synthesizes them into a clear, human-readable response for the user.

---

<img width="1200" src="https://storage.cloud.google.com/training-public-eikon/sql_pipeline_workflow.svg" alt="adk-mcp-diagram" />



## Step 1: Install and configure Toolbox

First, we need to download the latest version of the Google GenAI Toolbox binary and make it executable.

Download the [latest](https://github.com/googleapis/genai-toolbox/releases) version of Toolbox as a binary.

In [ ]:
version = "0.27.0" # x-release-please-version
! curl -O https://storage.googleapis.com/genai-toolbox/v{version}/linux/amd64/toolbox

# Make the binary executable
! chmod +x toolbox

In [ ]:
TOOLBOX_BINARY_PATH = "/content/toolbox"
SERVER_PORT = 5000

> Note: To include a literal dollar sign (e.g., $1) as part of your SQL statement within the Python string for tools.yml, you must escape both the backslash and the dollar sign. Use \\\$1 in Python to output \$1 in the tools.yml file.

> Note: You can also set up Colab secrets to store any sensitive information like passwords. You can easily add secrets through the left panel:

<img src="https://services.google.com/fh/files/misc/colab_secret.png" alt="Colab Secrets" width="400"/>


### 1.1 Secure Credentials & Tool Definition

> **Note:** We use Colab Secrets to safely store Snowflake credentials (`SNOWFLAKE_ACCOUNT`, `SNOWFLAKE_USERNAME`, etc.). Ensure your "Notebook access" is enabled in the Colab Secrets panel.

We will generate a `tools.yml` file that defines how our agents connect to Snowflake and the specific tools they are allowed to use (e.g., `execute-snowflake-query` and `list-tables`).

In [ ]:
import os
from google.colab import userdata

def load_colab_secrets() -> dict:
    """
    Load required Snowflake credentials from Colab Secrets
    into environment variables and return them as a dictionary.
    """

    required_keys = [
        "SNOWFLAKE_ACCOUNT",
        "SNOWFLAKE_USERNAME",
        "SNOWFLAKE_PASSWORD",
        "SNOWFLAKE_DATABASE",
        "SNOWFLAKE_SCHEMA",
        "SNOWFLAKE_WAREHOUSE",
        "SNOWFLAKE_ROLE"
    ]

    loaded_secrets = {}
    missing_keys = []

    print("🔄 Loading credentials from Colab Secrets...\n")

    for key in required_keys:
        try:
            value = userdata.get(key)

            if value is None or value == "":
                missing_keys.append(key)
                continue

            os.environ[key] = value
            loaded_secrets[key] = value
            print(f"✅ {key} loaded")

        except Exception as e:
            missing_keys.append(key)
            print(f"⚠️ Error loading {key}: {e}")

    if missing_keys:
        print("\n❌ Missing secrets:")
        for key in missing_keys:
            print(f"   - {key}")
        print("\nTip: Ensure 'Notebook access' is enabled in the Secrets panel.")

    else:
        print("\n🎉 All secrets successfully loaded.")

    return loaded_secrets

In [ ]:
secrets = load_colab_secrets()

SNOWFLAKE_ACCOUNT = secrets.get("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_USERNAME = secrets.get("SNOWFLAKE_USERNAME")
SNOWFLAKE_PASSWORD = secrets.get("SNOWFLAKE_PASSWORD")
SNOWFLAKE_DATABASE = secrets.get("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA = secrets.get("SNOWFLAKE_SCHEMA")
SNOWFLAKE_WAREHOUSE = secrets.get("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_ROLE = secrets.get("SNOWFLAKE_ROLE")

In [ ]:
# Create a tools file at runtime.
tools_file_name = "tools.yml"
file_content = f"""
sources:
  # Snowflake Source Configuration
  eikon-snowflake-source:
    kind: snowflake
    # The account identifier is the subdomain from your URL: wrtigml-fo42291
    account: {SNOWFLAKE_ACCOUNT}
    user: {SNOWFLAKE_USERNAME}
    password: {SNOWFLAKE_PASSWORD}
    database: {SNOWFLAKE_DATABASE}
    schema: {SNOWFLAKE_SCHEMA}
    warehouse: {SNOWFLAKE_WAREHOUSE}
    role: {SNOWFLAKE_ROLE}

tools:
  execute-snowflake-query:
    kind: snowflake-execute-sql
    source: eikon-snowflake-source
    description: >
      Directly execute any SQL statement against the EIKON database.
      Useful for complex joins, DDL, or exploring data structure not covered by specific tools.
      Do not use for production automated agents without review.

  list-tables:
    kind: snowflake-sql
    source: eikon-snowflake-source
    description: List all available tables in the database schema.
    statement: |
      SELECT TABLE_NAME, ROW_COUNT
      FROM INFORMATION_SCHEMA.TABLES
      WHERE TABLE_SCHEMA = CURRENT_SCHEMA();

toolsets:
  admin_toolset:
    - execute-snowflake-query
    - list-tables
"""

In [ ]:
# 1. Write the file safely using Python (Avoids shell escaping issues)
with open(tools_file_name, "w") as f:
    f.write(file_content)

In [ ]:
TOOLS_FILE_PATH = f"/content/{tools_file_name}"

In [ ]:
!pkill -f toolbox

### 1.2 Start the MCP Server
With our tools defined, we can start the local Toolbox server in the background.

In [ ]:
# Start a toolbox server
! nohup {TOOLBOX_BINARY_PATH} --ui --tools-file {TOOLS_FILE_PATH} -p {SERVER_PORT} > toolbox.log 2>&1 &
print(f"✅ Toolbox started on port {SERVER_PORT}")

In [ ]:
# Check if toolbox is running
!sudo lsof -i :{SERVER_PORT}

In [ ]:
# Print logs if it fails to ensure we see the error
! sleep 2 && cat toolbox.log

---

## Step 2: Connect your agent to Toolbox

To utilize Google's Vertex AI models, we must authenticate your Google Cloud Project.

> You need to authenticate as an IAM user so this notebook can access your Google Cloud Project. This access is necessary to use Google's LLM models.

In [ ]:
# Run this and allow access through the pop-up
from google.colab import auth

auth.authenticate_user()

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "your-project-id"  # @param {type:"string"}
location = 'us-central1' # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

> - [Connect using ADK](#scrollTo=QqRlWqvYNKSo)


### 2.1 Initialize ADK & Components
Here we install the Agent Development Kit (ADK), ChromaDB for vector storage, and configure our environment variables to route traffic through Vertex AI.

In [ ]:
! pip install google-adk[toolbox] toolbox-core chromadb --upgrade --quiet

In [ ]:
# environment configuration
import os

try:
    # The SDK uses this ID for usage tracking and billing
    os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

    # Defines the region where Vertex AI resources are hosted
    os.environ['GOOGLE_CLOUD_LOCATION'] = location

    # Directs the SDK to use Vertex AI infrastructure instead of the public Gemini API
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = "1"

    print(f"✅ Environment configured for project: {project_id} in {location}")

except Exception as e:
    print(f"❌ Configuration Error: {e}")

In [ ]:
# Import components
import os
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from pydantic import BaseModel, Field
from typing import List, Dict, Any

from google.adk.agents import LlmAgent, LoopAgent, SequentialAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools import FunctionTool
from google.adk.tools.tool_context import ToolContext
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.runners import InMemoryRunner

from google import genai
from google.genai import types
from google.genai.types import EmbedContentConfig

from toolbox_core import ToolboxSyncClient

`toolbox-core`: The client library that allows our Python agents to "talk" to the Toolbox server we started in Step 1.

---

## Step 3: Prepare and Ingest Vector Data

To give our LLM an understanding of our specific database, we need to provide it with the schema and some "golden" SQL examples. First, we will generate the dummy hospital data and save it locally.

In [ ]:
import json

schema_data = {
    "BILLING": {
        "description": "Financial records regarding patient treatments and insurance coverage.",
        "ddl": "create or replace TABLE BILLING (\n\tBILL_ID NUMBER(38,0) COMMENT 'Unique identifier for the bill.',\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the PATIENTS table.',\n\tHOSPITAL_ID NUMBER(38,0) COMMENT 'Foreign key referencing the HOSPITALS table.',\n\tAMOUNT NUMBER(38,0) COMMENT 'Total charged amount for the service.',\n\tPAID_AMOUNT NUMBER(38,0) COMMENT 'Amount paid out-of-pocket by the patient.',\n\tINSURANCE_COVERED NUMBER(38,0) COMMENT 'Amount covered by insurance providers (e.g., BPJS).',\n\tconstraint PK_BILLING primary key (BILL_ID)\n)COMMENT='Financial records regarding patient treatments and insurance coverage.'\n;",
        "sample_data": [
            {
                "BILL_ID": 643078,
                "PATIENT_ID": 76359,
                "HOSPITAL_ID": 5,
                "AMOUNT": 1334532,
                "PAID_AMOUNT": 463896,
                "INSURANCE_COVERED": 870636
            },
            {
                "BILL_ID": 643085,
                "PATIENT_ID": 79433,
                "HOSPITAL_ID": 2,
                "AMOUNT": 1033039,
                "PAID_AMOUNT": 167302,
                "INSURANCE_COVERED": 865737
            },
            {
                "BILL_ID": 643092,
                "PATIENT_ID": 96104,
                "HOSPITAL_ID": 3,
                "AMOUNT": 1878314,
                "PAID_AMOUNT": 811857,
                "INSURANCE_COVERED": 1066457
            }
        ]
    },
    "HOSPITALS": {
        "description": "Master list of hospital facilities.",
        "ddl": "create or replace TABLE HOSPITALS (\n\tHOSPITAL_ID NUMBER(38,0) COMMENT 'Unique identifier for the hospital.',\n\tNAME VARCHAR(16777216) COMMENT 'Official name of the hospital.',\n\tCITY VARCHAR(16777216) COMMENT 'City where the hospital is located.',\n\tconstraint PK_HOSPITALS primary key (HOSPITAL_ID),\n\tconstraint UK_HOSPITALS_NAME unique (NAME)\n)COMMENT='Master list of hospital facilities.'\n;",
        "sample_data": [
            {
                "HOSPITAL_ID": 1,
                "NAME": "RS Jakarta Pusat",
                "CITY": "Jakarta"
            },
            {
                "HOSPITAL_ID": 2,
                "NAME": "RS Surabaya Timur",
                "CITY": "Surabaya"
            },
            {
                "HOSPITAL_ID": 3,
                "NAME": "RS Bandung Sehat",
                "CITY": "Bandung"
            }
        ]
    },
    "DOCTORS": {
        "description": "Registry of doctors, their specialties, and hospital affiliations.",
        "ddl": "create or replace TABLE DOCTORS (\n\tDOCTOR_ID NUMBER(38,0) COMMENT 'Unique identifier for the doctor.',\n\tHOSPITAL_ID NUMBER(38,0) COMMENT 'Foreign key referencing the HOSPITALS table.',\n\tNAME VARCHAR(16777216) COMMENT 'Full name of the doctor.',\n\tSPECIALTY VARCHAR(16777216) COMMENT 'Medical specialization (e.g., Pediatrics, Orthopedics).',\n\tconstraint PK_DOCTORS primary key (DOCTOR_ID)\n)COMMENT='Registry of doctors, their specialties, and hospital affiliations.'\n;",
        "sample_data": [
            {
                "DOCTOR_ID": 1,
                "HOSPITAL_ID": 4,
                "NAME": "Budi Santoso",
                "SPECIALTY": "Neurology"
            },
            {
                "DOCTOR_ID": 2,
                "HOSPITAL_ID": 5,
                "NAME": "Rina Pratama",
                "SPECIALTY": "Orthopedic"
            },
            {
                "DOCTOR_ID": 3,
                "HOSPITAL_ID": 3,
                "NAME": "Siti Saputra",
                "SPECIALTY": "Pediatrics"
            }
        ]
    },
    "APPOINTMENTS": {
        "description": "Scheduled meetings between patients and doctors.",
        "ddl": "create or replace TABLE APPOINTMENTS (\n\tAPPOINTMENT_ID NUMBER(38,0) COMMENT 'Unique identifier for the appointment.',\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the PATIENTS table.',\n\tDOCTOR_ID NUMBER(38,0) COMMENT 'Foreign key referencing the DOCTORS table.',\n\tDATE DATE COMMENT 'Scheduled date of the appointment.',\n\tconstraint PK_APPOINTMENTS primary key (APPOINTMENT_ID)\n)COMMENT='Scheduled meetings between patients and doctors.'\n;",
        "sample_data": [
            {
                "APPOINTMENT_ID": 3001,
                "PATIENT_ID": 1001,
                "DOCTOR_ID": 101,
                "DATE": "2026-02-01"
            },
            {
                "APPOINTMENT_ID": 3002,
                "PATIENT_ID": 1002,
                "DOCTOR_ID": 102,
                "DATE": "2026-02-02"
            },
            {
                "APPOINTMENT_ID": 3003,
                "PATIENT_ID": 1003,
                "DOCTOR_ID": 103,
                "DATE": "2026-02-03"
            }
        ]
    },
    "ICD10_CODES": {
        "description": "Reference table for International Classification of Diseases (ICD-10).",
        "ddl": "create or replace TABLE ICD10_CODES (\n\tC1 VARCHAR(16777216) COMMENT 'The ICD-10 alphanumeric code (e.g., I10, E11).',\n\tC2 VARCHAR(16777216) COMMENT 'The official description/diagnosis for the code.',\n\tconstraint PK_ICD10 primary key (C1)\n)COMMENT='Reference table for International Classification of Diseases (ICD-10).'\n;",
        "sample_data": [
            {
                "C1": "icd10_code",
                "C2": "description"
            },
            {
                "C1": "I10",
                "C2": "Essential hypertension"
            },
            {
                "C1": "E11",
                "C2": "Type 2 diabetes mellitus"
            }
        ]
    },
    "MEDICATIONS": {
        "description": "Master catalog of available pharmaceutical drugs.",
        "ddl": "create or replace TABLE MEDICATIONS (\n\tMEDICATION_ID VARCHAR(16777216) COMMENT 'Unique alphanumeric identifier for the medication.',\n\tNAME VARCHAR(16777216) COMMENT 'Commercial or generic name of the medication.',\n\tconstraint PK_MEDICATIONS primary key (MEDICATION_ID),\n\tconstraint UK_MEDICATIONS_NAME unique (NAME)\n)COMMENT='Master catalog of available pharmaceutical drugs.'\n;",
        "sample_data": [
            {
                "MEDICATION_ID": "MED001",
                "NAME": "Paracetamol"
            },
            {
                "MEDICATION_ID": "MED002",
                "NAME": "Amoxicillin"
            },
            {
                "MEDICATION_ID": "MED003",
                "NAME": "Amlodipine"
            }
        ]
    },
    "ADMISSIONS": {
        "description": "Records of patient admissions to hospital rooms.",
        "ddl": "create or replace TABLE ADMISSIONS (\n\tADMISSION_ID NUMBER(38,0) COMMENT 'Unique identifier for the admission event.',\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the PATIENTS table.',\n\tROOM_ID NUMBER(38,0) COMMENT 'Foreign key referencing the ROOMS table.',\n\tDOCTOR_ID NUMBER(38,0) COMMENT 'Foreign key referencing the attending DOCTOR.',\n\tADMIT_DATE DATE COMMENT 'The date the patient was admitted to the hospital.',\n\tconstraint PK_ADMISSIONS primary key (ADMISSION_ID)\n)COMMENT='Records of patient admissions to hospital rooms.'\n;",
        "sample_data": [
            {
                "ADMISSION_ID": 4001,
                "PATIENT_ID": 1001,
                "ROOM_ID": 201,
                "DOCTOR_ID": 101,
                "ADMIT_DATE": "2026-02-01"
            },
            {
                "ADMISSION_ID": 4002,
                "PATIENT_ID": 1002,
                "ROOM_ID": 202,
                "DOCTOR_ID": 102,
                "ADMIT_DATE": "2026-02-02"
            }
        ]
    },
    "DEPARTMENTS": {
        "description": "List of medical departments within the hospital network.",
        "ddl": "create or replace TABLE DEPARTMENTS (\n\tDEPARTMENT_ID NUMBER(38,0) COMMENT 'Unique identifier for the department.',\n\tNAME VARCHAR(16777216) COMMENT 'Name of the department (e.g., Cardiology, Neurology).',\n\tconstraint PK_DEPARTMENTS primary key (DEPARTMENT_ID),\n\tconstraint UK_DEPARTMENTS_NAME unique (NAME)\n)COMMENT='List of medical departments within the hospital network.'\n;",
        "sample_data": [
            {
                "DEPARTMENT_ID": 1,
                "NAME": "Cardiology"
            },
            {
                "DEPARTMENT_ID": 2,
                "NAME": "Neurology"
            },
            {
                "DEPARTMENT_ID": 3,
                "NAME": "Orthopedics"
            }
        ]
    },
    "INSURANCE": {
        "description": "Details of insurance providers and coverage classes for patients.",
        "ddl": "create or replace TABLE INSURANCE (\n\tINSURANCE_ID NUMBER(38,0) COMMENT 'Unique identifier for the insurance record.',\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the PATIENTS table.',\n\tPROVIDER VARCHAR(16777216) COMMENT 'Name of the insurance provider (e.g., BPJS).',\n\tCLASS VARCHAR(16777216) COMMENT 'Service class level (e.g., Class 1, Class 2, Class 3).',\n\tconstraint PK_INSURANCE primary key (INSURANCE_ID),\n\tconstraint UK_INSURANCE_PATIENT_PROVIDER unique (PATIENT_ID, PROVIDER)\n)COMMENT='Details of insurance providers and coverage classes for patients.'\n;",
        "sample_data": [
            {
                "INSURANCE_ID": 1,
                "PATIENT_ID": 1,
                "PROVIDER": "BPJS",
                "CLASS": "Class 3"
            },
            {
                "INSURANCE_ID": 2,
                "PATIENT_ID": 2,
                "PROVIDER": "BPJS",
                "CLASS": "Class 2"
            },
            {
                "INSURANCE_ID": 3,
                "PATIENT_ID": 3,
                "PROVIDER": "BPJS",
                "CLASS": "Class 3"
            }
        ]
    },
    "LAB_RESULTS": {
        "description": "Results of laboratory tests performed on patients.",
        "ddl": "create or replace TABLE LAB_RESULTS (\n\tLAB_ID NUMBER(38,0) COMMENT 'Unique identifier for the laboratory test record.',\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the PATIENTS table.',\n\tHOSPITAL_ID NUMBER(38,0) COMMENT 'Foreign key referencing the HOSPITALS table.',\n\tTEST VARCHAR(16777216) COMMENT 'Type of test performed (e.g., MRI, Urine).',\n\tRESULT VARCHAR(16777216) COMMENT 'Outcome or observation of the test (e.g., Normal, Abnormal).',\n\tconstraint PK_LAB_RESULTS primary key (LAB_ID)\n)COMMENT='Results of laboratory tests performed on patients.'\n;",
        "sample_data": [
            {
                "LAB_ID": 1,
                "PATIENT_ID": 6437,
                "HOSPITAL_ID": 3,
                "TEST": "MRI",
                "RESULT": "Abnormal"
            },
            {
                "LAB_ID": 2,
                "PATIENT_ID": 71316,
                "HOSPITAL_ID": 3,
                "TEST": "Urine",
                "RESULT": "Normal"
            },
            {
                "LAB_ID": 3,
                "PATIENT_ID": 27487,
                "HOSPITAL_ID": 3,
                "TEST": "Urine",
                "RESULT": "Abnormal"
            }
        ]
    },
    "MEDICAL_RECORDS": {
        "description": "Clinical records linking patients to specific diagnoses.",
        "ddl": "create or replace TABLE MEDICAL_RECORDS (\n\tRECORD_ID NUMBER(38,0) COMMENT 'Unique identifier for the medical record entry.',\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the PATIENTS table.',\n\tICD10_CODE VARCHAR(16777216) COMMENT 'Foreign key referencing ICD10_CODES (Diagnosis).',\n\tconstraint PK_MEDICAL_RECORDS primary key (RECORD_ID)\n)COMMENT='Clinical records linking patients to specific diagnoses.'\n;",
        "sample_data": [
            {
                "RECORD_ID": 1,
                "PATIENT_ID": 65067,
                "ICD10_CODE": "M54"
            },
            {
                "RECORD_ID": 2,
                "PATIENT_ID": 29491,
                "ICD10_CODE": "I10"
            },
            {
                "RECORD_ID": 3,
                "PATIENT_ID": 41735,
                "ICD10_CODE": "J18"
            }
        ]
    },
    "ROOMS": {
        "description": "Physical rooms located within hospital departments.",
        "ddl": "create or replace TABLE ROOMS (\n\tROOM_ID NUMBER(38,0) COMMENT 'Unique identifier for the room entity.',\n\tROOM_NUMBER VARCHAR(16777216) COMMENT 'Visible room number/label (e.g., A101).',\n\tDEPARTMENT_ID NUMBER(38,0) COMMENT 'Foreign key referencing the DEPARTMENTS table.',\n\tconstraint PK_ROOMS primary key (ROOM_ID),\n\tconstraint UK_ROOMS_DEPT_NUM unique (DEPARTMENT_ID, ROOM_NUMBER)\n)COMMENT='Physical rooms located within hospital departments.'\n;",
        "sample_data": [
            {
                "ROOM_ID": 201,
                "ROOM_NUMBER": "A101",
                "DEPARTMENT_ID": 1
            },
            {
                "ROOM_ID": 202,
                "ROOM_NUMBER": "B201",
                "DEPARTMENT_ID": 2
            },
            {
                "ROOM_ID": 203,
                "ROOM_NUMBER": "C301",
                "DEPARTMENT_ID": 3
            }
        ]
    },
    "PATIENTS": {
        "description": "Demographic information for patients.",
        "ddl": "create or replace TABLE PATIENTS (\n\tPATIENT_ID NUMBER(38,0) COMMENT 'Unique identifier for the patient.',\n\tNAME VARCHAR(16777216) COMMENT 'Full name of the patient.',\n\tGENDER VARCHAR(16777216) COMMENT 'Gender of the patient (M/F).',\n\tBIRTH_YEAR NUMBER(38,0) COMMENT 'Year of birth of the patient.',\n\tBPJS_NUMBER NUMBER(38,0) COMMENT 'Indonesian National Health Insurance (BPJS Kesehatan) number.',\n\tconstraint PK_PATIENTS primary key (PATIENT_ID),\n\tconstraint UK_PATIENTS_BPJS unique (BPJS_NUMBER)\n)COMMENT='Demographic information for patients.'\n;",
        "sample_data": [
            {
                "PATIENT_ID": 1,
                "NAME": "Siti Hidayat",
                "GENDER": "M",
                "BIRTH_YEAR": 1948,
                "BPJS_NUMBER": 848493407578
            },
            {
                "PATIENT_ID": 2,
                "NAME": "Dedi Wijaya",
                "GENDER": "F",
                "BIRTH_YEAR": 2009,
                "BPJS_NUMBER": 693778197140
            },
            {
                "PATIENT_ID": 3,
                "NAME": "Lina Wijaya",
                "GENDER": "M",
                "BIRTH_YEAR": 1963,
                "BPJS_NUMBER": 994597173056
            }
        ]
    },
    "PHARMACY_INVENTORY": {
        "description": "Current stock levels of medications in the pharmacy.",
        "ddl": "create or replace TABLE PHARMACY_INVENTORY (\n\tINVENTORY_ID NUMBER(38,0) COMMENT 'Unique identifier for the inventory record.',\n\tMEDICATION_ID VARCHAR(16777216) COMMENT 'Foreign key referencing the MEDICATIONS table.',\n\tSTOCK NUMBER(38,0) COMMENT 'Quantity of medication currently available.',\n\tconstraint PK_PHARMACY_INVENTORY primary key (INVENTORY_ID),\n\tconstraint UK_PHARMACY_MEDICATION unique (MEDICATION_ID)\n)COMMENT='Current stock levels of medications in the pharmacy.'\n;",
        "sample_data": [
            {
                "INVENTORY_ID": 1,
                "MEDICATION_ID": "MED001",
                "STOCK": 5324
            },
            {
                "INVENTORY_ID": 2,
                "MEDICATION_ID": "MED002",
                "STOCK": 5634
            },
            {
                "INVENTORY_ID": 3,
                "MEDICATION_ID": "MED003",
                "STOCK": 6825
            }
        ]
    },
    "BEDS": {
        "description": "Inventory of hospital beds including type and current operational status.",
        "ddl": "create or replace TABLE BEDS (\n\tBED_ID NUMBER(38,0) COMMENT 'Unique identifier for the bed.',\n\tHOSPITAL_ID NUMBER(38,0) COMMENT 'Foreign key referencing the HOSPITALS table.',\n\tBED_TYPE VARCHAR(16777216) COMMENT 'Classification of the bed (e.g., ICU, VIP, Regular).',\n\tSTATUS VARCHAR(16777216) COMMENT 'Current availability status (e.g., Maintenance, Occupied, Available).',\n\tconstraint PK_BEDS primary key (BED_ID)\n)COMMENT='Inventory of hospital beds including type and current operational status.'\n;",
        "sample_data": [
            {
                "BED_ID": 1,
                "HOSPITAL_ID": 1,
                "BED_TYPE": "ICU",
                "STATUS": "Maintenance"
            },
            {
                "BED_ID": 2,
                "HOSPITAL_ID": 1,
                "BED_TYPE": "ICU",
                "STATUS": "Maintenance"
            },
            {
                "BED_ID": 3,
                "HOSPITAL_ID": 1,
                "BED_TYPE": "VIP",
                "STATUS": "Maintenance"
            }
        ]
    },
    "NURSES": {
        "description": "Registry of nursing staff and their hospital assignments.",
        "ddl": "create or replace TABLE NURSES (\n\tNURSE_ID NUMBER(38,0) COMMENT 'Unique identifier for the nurse.',\n\tHOSPITAL_ID NUMBER(38,0) COMMENT 'Foreign key referencing the HOSPITALS table.',\n\tNAME VARCHAR(16777216) COMMENT 'Full name of the nurse.',\n\tconstraint PK_NURSES primary key (NURSE_ID)\n)COMMENT='Registry of nursing staff and their hospital assignments.'\n;",
        "sample_data": [
            {
                "NURSE_ID": 1,
                "HOSPITAL_ID": 2,
                "NAME": "Budi Gunawan"
            },
            {
                "NURSE_ID": 2,
                "HOSPITAL_ID": 1,
                "NAME": "Dewi Saputra"
            },
            {
                "NURSE_ID": 3,
                "HOSPITAL_ID": 1,
                "NAME": "Dewi Hidayat"
            }
        ]
    },
    "PRESCRIPTIONS": {
        "description": "Medications prescribed to patients linked to medical records.",
        "ddl": "create or replace TABLE PRESCRIPTIONS (\n\tPRESCRIPTION_ID NUMBER(38,0) COMMENT 'Unique identifier for the prescription.',\n\tRECORD_ID NUMBER(38,0) COMMENT 'Foreign key referencing MEDICAL_RECORDS.',\n\tMEDICATION_ID VARCHAR(16777216) COMMENT 'Foreign key referencing the MEDICATIONS table.',\n\tQUANTITY NUMBER(38,0) COMMENT 'Amount or dosage of medication prescribed.',\n\tconstraint PK_PRESCRIPTIONS primary key (PRESCRIPTION_ID)\n)COMMENT='Medications prescribed to patients linked to medical records.'\n;",
        "sample_data": [
            {
                "PRESCRIPTION_ID": 1,
                "RECORD_ID": 393983,
                "MEDICATION_ID": "MED003",
                "QUANTITY": 26
            },
            {
                "PRESCRIPTION_ID": 2,
                "RECORD_ID": 498734,
                "MEDICATION_ID": "MED004",
                "QUANTITY": 2
            },
            {
                "PRESCRIPTION_ID": 3,
                "RECORD_ID": 98789,
                "MEDICATION_ID": "MED001",
                "QUANTITY": 19
            }
        ]
    }
}



In [ ]:
examples_data = [
  {
    "id": "1",
    "question": "What is the total charged billing amount for each hospital, ordered from highest to lowest?",
    "sql": "SELECT h.NAME AS HOSPITAL_NAME, SUM(b.AMOUNT) AS TOTAL_AMOUNT FROM HOSPITALS h JOIN BILLING b ON h.HOSPITAL_ID = b.HOSPITAL_ID GROUP BY h.NAME ORDER BY TOTAL_AMOUNT DESC;"
  },
  {
    "id": "2",
    "question": "How many doctors are currently registered under each medical specialty across the entire hospital network?",
    "sql": "SELECT SPECIALTY, COUNT(DOCTOR_ID) AS DOCTOR_COUNT FROM DOCTORS GROUP BY SPECIALTY ORDER BY DOCTOR_COUNT DESC;"
  },
  {
    "id": "3",
    "question": "What is the average out-of-pocket amount paid by patients grouped by their insurance provider?",
    "sql": "SELECT i.PROVIDER AS INSURANCE_PROVIDER, AVG(b.PAID_AMOUNT) AS AVG_PAID_AMOUNT FROM INSURANCE i JOIN BILLING b ON i.PATIENT_ID = b.PATIENT_ID GROUP BY i.PROVIDER;"
  },
  {
    "id": "4",
    "question": "Which doctor has handled the highest number of patient admissions, and how many admissions did they handle?",
    "sql": "SELECT d.NAME AS DOCTOR_NAME, COUNT(a.ADMISSION_ID) AS ADMISSION_COUNT FROM DOCTORS d JOIN ADMISSIONS a ON d.DOCTOR_ID = a.DOCTOR_ID GROUP BY d.NAME ORDER BY ADMISSION_COUNT DESC LIMIT 1;"
  },
  {
    "id": "5",
    "question": "What are the top 5 most frequently recorded diagnoses (by their official description) in the medical records?",
    "sql": "SELECT c.C2 AS DIAGNOSIS_DESCRIPTION, COUNT(m.RECORD_ID) AS RECORD_COUNT FROM ICD10_CODES c JOIN MEDICAL_RECORDS m ON c.C1 = m.ICD10_CODE GROUP BY c.C2 ORDER BY RECORD_COUNT DESC LIMIT 5;"
  }
]

In [ ]:
with open("snowflake_schema_dump.json", "w") as f:
    json.dump(schema_data, f)

with open("sql_examples.json", "w") as f:
    json.dump(examples_data, f)

print("✅ Dummy data files created.")

### 3.1 Ingest Data into ChromaDB

We use `gemini-embedding-001` to embed our schema definitions and SQL examples. This allows our retrieval agents to perform semantic searches later—finding the right tables based on the meaning of the user's question, not just exact keyword matches.

In [ ]:
CHROMA_PATH = "./chroma_db"
SCHEMA_FILE = "snowflake_schema_dump.json"
EXAMPLES_FILE = "sql_examples.json"

In [ ]:
# 1. Setup Embedding Function
class GeminiEmbeddingFunction(EmbeddingFunction):
    """Custom Embedding function to let Chroma use Google GenAI."""
    def __init__(self):
        self.client = genai.Client()

    def __call__(self, input: Documents) -> Embeddings:
        response = self.client.models.embed_content(
            model="gemini-embedding-001",
            contents=input,
            config=types.EmbedContentConfig(
                task_type="RETRIEVAL_DOCUMENT",
                output_dimensionality=768
            ),
        )
        return [e.values for e in response.embeddings]

# Initialize Client
print(f"Initializing ChromaDB at {CHROMA_PATH}...")
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)
embedding_fn = GeminiEmbeddingFunction()

In [ ]:
# 2. Ingestion Logic: SQL Examples
def ingest_sql_examples():
    collection_name = "sql_examples"
    print(f"--- Processing {collection_name} ---")

    if not os.path.exists(EXAMPLES_FILE):
        print(f"Error: {EXAMPLES_FILE} not found.")
        return

    with open(EXAMPLES_FILE, 'r') as f:
        data = json.load(f)

    collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embedding_fn
    )

    ids = []
    documents = []
    metadatas = []

    for item in data:
        ids.append(item["id"])
        documents.append(item["question"])
        metadatas.append({"sql": item["sql"]})

    if ids:
        collection.upsert(ids=ids, documents=documents, metadatas=metadatas)
        print(f"Successfully ingested {len(ids)} items into '{collection_name}'.")
    else:
        print("No items found to ingest.")

In [ ]:
# 3. Ingest Schema / DDL
def ingest_schema():
    collection_name = "database_schema"
    print(f"--- Processing {collection_name} ---")

    if not os.path.exists(SCHEMA_FILE):
        print(f"Error: {SCHEMA_FILE} not found.")
        return

    with open(SCHEMA_FILE, 'r') as f:
        schema_data = json.load(f)

    collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embedding_fn
    )

    ids = []
    documents = []
    metadatas = []

    for table_name, info in schema_data.items():
        description = info.get("description", "")
        ddl = info.get("ddl", "")

        # fully qualified name
        fqn = f"{SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.{table_name}"

        semantic_doc = (
            f"Table: {fqn}\n"
            f"Database: {SNOWFLAKE_DATABASE}\n"
            f"Schema: {SNOWFLAKE_SCHEMA}\n"
            f"Description: {description}\n"
            f"Definition: {ddl}"
        )

        ids.append(table_name)
        documents.append(semantic_doc)

        metadatas.append({
            "table_name": table_name,
            "database": SNOWFLAKE_DATABASE,
            "schema": SNOWFLAKE_SCHEMA,
            "fully_qualified_name": fqn,
            "description": description,
            "ddl": ddl
        })

    if ids:
        collection.upsert(ids=ids, documents=documents, metadatas=metadatas)
        print(f"Successfully ingested {len(ids)} tables into '{collection_name}'.")
    else:
        print("No tables found to ingest.")

In [ ]:
# Run Ingestion
try:
    ingest_sql_examples()
    ingest_schema()
    print("\nIngestion Complete.")
except Exception as e:
    print(f"\nAn error occurred: {e}")

---

## Step 4: Define Agents & The Pipeline

### 4.1 Define Data Models & Vector Search Tools
We use Pydantic `BaseModels` to force our agents to output strictly structured JSON. We also define the specific python functions (`search_vector_db` and `search_schema_db`) that our agents will call to interact with ChromaDB.

In [ ]:
# Configure Retry Options
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)


In [ ]:
server_url="http://127.0.0.1:5000"

toolbox = ToolboxSyncClient(server_url)

# load only execute SQL tool
execute_sql_tool = toolbox.load_tool('execute-snowflake-query')
print("✅ MCP Tool created")

In [ ]:
class SqlExample(BaseModel):
    question: str = Field(description="The natural language question associated with the SQL example.")
    query: str = Field(description="The SQL query corresponding to the question.")
    similarity: float = Field(description="The similarity score (0-1) indicating how relevant this example is.")

class KnownSqlOutput(BaseModel):
    user_query: str = Field(description="The user's original natural language query.")
    sql_examples: List[SqlExample] = Field(
        description="A list of structured examples containing both the original question and the verified SQL query."
    )

In [ ]:
class TableInfo(BaseModel):
    table_name: str = Field(description="The fully qualified name of the table.")
    ddl: str = Field(description="The CREATE TABLE statement.")
    description: str = Field(description="Description of the table's contents.")
    similarity: float

class KnownSchemaOutput(BaseModel):
    user_query: str = Field(description="The user's original natural language query.")
    relevant_tables: List[TableInfo] = Field(description="List of relevant tables found in the database schema.")

In [ ]:
class SqlOutput(BaseModel):
    sql: str = Field(description="The valid Snowflake SQL query text.")

In [ ]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    """Custom Embedding function to let Chroma use Google GenAI."""
    def __init__(self):
      self.client = genai.Client()

    def __call__(self, input: Documents) -> Embeddings:
        client = genai.Client()
        response = client.models.embed_content(
            model="gemini-embedding-001",
            contents=input,
            config=types.EmbedContentConfig(
                task_type="RETRIEVAL_QUERY",
                output_dimensionality=768
            ),
        )
        return [e.values for e in response.embeddings]

In [ ]:
examples_collection = chroma_client.get_collection(
    name="sql_examples",
    embedding_function=GeminiEmbeddingFunction()
)

schema_collection = chroma_client.get_collection(
    name="database_schema",
    embedding_function=GeminiEmbeddingFunction()
)

In [ ]:
def search_vector_db(query: str) -> List[Dict[str, Any]]:
    print(f"  [Vector DB] Searching Examples for: {query}")
    results = examples_collection.query(query_texts=[query], n_results=3)
    found_examples = []

    if results['documents']:
        for i in range(len(results['documents'][0])):
            doc = results['documents'][0][i]
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i]
            similarity = 1.0 - distance

            # Lowered threshold for demo purposes; orig was 0.80
            if similarity > 0.60:
                found_examples.append({
                    "question": doc,
                    "sql": metadata['sql'],
                    "database": metadata.get('database', 'EIKON'),
                    "schema": metadata.get('schema', 'DEMO'),
                    "similarity": round(similarity, 4)
                })
            else:
                print(f"  [Vector DB] Skipped result (Similarity {similarity:.2f})")
    return found_examples

retrieve_sql_tool = FunctionTool(func=search_vector_db)

In [ ]:
def search_schema_db(query: str) -> List[Dict[str, Any]]:
    print(f"  [Vector DB] Searching Schema for: {query}")
    results = schema_collection.query(query_texts=[query], n_results=5)
    found_tables = []

    if results['documents']:
        for i in range(len(results['documents'][0])):
            metadata = results['metadatas'][0][i]
            distance = results['distances'][0][i]
            similarity = 1.0 - distance


            found_tables.append({
                "table_name": metadata.get('fully_qualified_name', metadata.get('table_name')),
                "description": metadata.get('description'),
                "ddl": metadata.get('ddl'),
                "similarity": round(similarity, 4)
            })

    return found_tables

retrieve_schema_tool = FunctionTool(func=search_schema_db)

In [ ]:
def exit_loop(tool_context: ToolContext):
    print(f"  [Loop] Validation passed. Exiting loop.")
    tool_context.actions.escalate = True
    return {}


### 4.2 The Retrieval Stage (Parallel Agents)
Instead of fetching context sequentially, we deploy two agents in a `ParallelAgent` structure. One agent hunts for relevant tables (DDL), while the other hunts for helpful query templates simultaneously. This saves time and provides the generation agent with a rich context payload.

In [ ]:
sql_retrieval_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="SqlRetrievalAgent",
    description=(
        "This agent accesses a Vector Database to find previously vetted, 'good known' SQL queries. "
        "It is useful for finding correct JOIN patterns or complex logic examples based on a natural language description."
    ),
    instruction=(
        "You are an expert SQL assistant. Your goal is to find valid SQL examples that match the user's intent. "
        "1. Use the 'search_vector_db' tool to find similar SQL queries. "
        "2. Review the results to ensure they are relevant to the user's request. "
        "3. Return the list of SQL queries in the specified JSON format."
        "CRITICAL: The 'user_query' field in the output MUST be an exact copy of the input. Do not rewrite, refine, or interpret it."
    ),
    tools=[retrieve_sql_tool],
    output_schema=KnownSqlOutput,
    output_key="retrieved_sql_context"
)

schema_retrieval_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="SchemaRetrievalAgent",
    description="Accesses the Vector Database to find relevant Database Tables and DDL.",
    instruction=(
        "You are a Database Architect assistant. Your goal is to find the specific database tables needed to answer the user's question. "
        "1. Use the 'search_schema_db' tool to find relevant tables based on keywords in the user query. "
        "2. Return the list of tables including their DDL and descriptions. "
        "CRITICAL: The 'user_query' field in the output MUST be an exact copy of the input."
    ),
    tools=[retrieve_schema_tool],
    output_schema=KnownSchemaOutput,
    output_key="retrieved_schema_context"
)

retrieval_stage = ParallelAgent(
    name="RetrievalStage",
    description="Retrieves both similar SQL examples and relevant Schema definitions in parallel.",
    sub_agents=[sql_retrieval_agent, schema_retrieval_agent]
)

### 4.3 SQL Generation & Self-Healing Loop
This is the core of the pipeline.
1. The **Generation Agent** writes a draft query based on the retrieved context.
2. The **Validation Agent** critiques the query for syntax and security (preventing drops/deletes).
3. If an error is found, the **Refiner Agent** rewrites it.

This happens inside a `LoopAgent` that runs up to 3 times or until the query is perfect.

In [ ]:
sql_generation_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="SqlGenerationAgent",
    description="Generates the initial SQL query.",
    instruction="""
    You are a Snowflake SQL Expert.

    **User Request:**

    **1. Relevant SQL Examples (Reference for Logic/Joins):**
    {retrieved_sql_context}

    **2. Database Schema (Reference for Table Names/Columns):**
    {retrieved_schema_context}

    **Task:**
    Generate a valid Snowflake SQL query for the User Request.

    **Rules:**
    - **Schema Priority:** ALWAYS use the Table Names and Column definitions found in the `Database Schema`.
    - **Logic:** Use the `SQL Examples` to understand how to join these tables (e.g., how to link Patients to Admissions).
    - **Safety:** Default to `LIMIT 10` if the user asks for a list.

    **Output:**
    Populate the `sql` field in the output schema.
    """,
    output_schema=SqlOutput,
    output_key="generated_sql"
    )

In [ ]:
sql_validation_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="SqlValidationAgent",
    instruction="""
    You are a SQL Security & Syntax Validator.

    **SQL to Check:**
    {generated_sql}

    **Checklist:**
    1. Is it valid Snowflake SQL?
    2. Is it SAFE? (No DROP, DELETE, INSERT, UPDATE).
    3. Does it answer the user's specific request?

    **Output:**
    - If PERFECT: Respond exactly "No major issues found."
    - If ISSUES: Provide a short, bulleted list of what to fix.
    """,
    output_key="sql_criticism"
)

sql_refiner_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="SqlRefinerAgent",
    instruction="""
    You are a SQL Editor.

    **Current SQL:** {generated_sql}
    **Critique:** {sql_criticism}

    **Task:**
    1. IF critique says "No major issues found.":
       - Call the 'exit_loop' tool.
       - **CRITICAL:** You MUST output the **Current SQL** exactly as is. Do not output the critique text.

    2. ELSE (If there are issues):
       - Rewrite the SQL to fix the errors.
       - Populate the `sql` field with the fixed SQL.
    """,
    tools=[exit_loop],
    output_schema=SqlOutput,
    output_key="generated_sql"
)

sql_validation_loop = LoopAgent(
    name="SqlValidationLoop",
    sub_agents=[sql_validation_agent, sql_refiner_agent],
    max_iterations=3
)

### 4.4 Execution & Insight Synthesis
Once the SQL is validated, the **Executor Agent** runs the code securely against Snowflake using our Toolbox. Finally, the **Insight Generator Agent** takes the raw database rows and formats them into a polite, easy-to-read natural language summary for the user.

In [ ]:
sql_executor_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="SqlExecutorAgent",
    description="Executes the validated SQL.",
    instruction="""
    You are a Database Runner.

    **SQL:** {generated_sql}

    **Task:**
    1. Execute the SQL using 'execute-snowflake-query'.
    2. Return the raw JSON/Text results.
    """,
    tools=[execute_sql_tool],
    output_key="query_results"
)

insight_generator_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name="InsightGeneratorAgent",
    description="Explains the data to the user.",
    instruction="""
    You are a Data Analyst.

    **User Question:**
    **Raw Data:** {query_results}

    **Task:**
    Provide a natural language answer to the user's question based on the Raw Data.
    - Summarize lists.
    - Highlight key numbers.
    - If data is empty, politely inform the user.
    """,
)

### 4.5 Orchestrating the Chat Pipeline
We wrap the entire process (Retrieval ➔ Generate ➔ Validate ➔ Execute ➔ Insight) into a `SequentialAgent`.

We then give this entire pipeline to our **Root Agent** as a tool. The Root Agent acts as the front door, deciding whether to make small talk or trigger the massive data pipeline. Let's run a test query!

In [ ]:
chat_pipeline = SequentialAgent(
    name="ChatWithDataPipeline",
    description="Retrieves context, generates SQL, validates, executes, and explains.",
    sub_agents=[
        retrieval_stage,
        sql_generation_agent,
        sql_validation_loop,
        sql_executor_agent,
        insight_generator_agent
    ]
)

sql_pipeline_tool = AgentTool(agent=chat_pipeline, skip_summarization=False)

In [ ]:
root_agent = LlmAgent(
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_config),
    name='root_agent',
    description='The primary interface for the hospital assistant. It routes data-related questions to the SQL pipeline and handles general inquiries directly.',
    instruction="""
    You are a helpful Hospital Data Assistant.

    **Your Goal:**
    Determine if the user's request requires accessing the database or if it is a general conversation.

    **Rules for Tool Usage:**
    1. **Data Requests:** IF the user asks for specific information, lists, counts, or statistics, you MUST use the `sql_pipeline_tool`.
       - Pass the user's *exact* natural language query as the input to the tool.
       - Do NOT attempt to generate SQL or answer data questions from your own internal knowledge.

    2. **General Chat:** IF the user offers a greeting (e.g., "Hello", "Hi") or asks about your identity, answer politely and briefly without using the tool.

    3. **Out of Scope:** IF the user asks questions unrelated to hospital data (e.g., "What is the capital of France?"), politely inform them that you can only assist with hospital database queries.
    """,
    tools=[
        sql_pipeline_tool
    ]
)

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Which doctor has handled the highest number of patient admissions, and how many admissions did they handle?"
)

---